## Importamos Bibliotecas

In [4]:
# Biblioteca pandas para el manejo de datos
import pandas as pd

# EXTRACCIÓN DE LOS DATOS


In [139]:
# Extraemos la información de cada csv y se convierte en un dataframe
canales = pd.read_csv('..\\data\\canales.csv', encoding='utf-8')
clientes = pd.read_csv('..\\data\\clientes.csv', encoding='utf-8')
transacciones = pd.read_csv('..\\data\\transacciones.csv', encoding='utf-8')

# TRANSFORMACIÓN DE LOS DATOS 

In [140]:
# Eliminamos valores duplicados para canales teniendo en cuenta la columna 'codigo'
canales.drop_duplicates(subset=['codigo'],inplace=True)
# Eliminamos valores duplicados para clientes teniendo en cuenta la columna 'num_doc'
clientes.drop_duplicates(subset=['num_doc'],inplace=True)

In [141]:
canales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32974 entries, 0 to 32973
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo            32974 non-null  int64 
 1   nombre            32974 non-null  object
 2   tipo              32974 non-null  object
 3   cod_jurisdiccion  32974 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ MB


In [142]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 698022 entries, 0 to 701051
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tipo_doc            698022 non-null  object 
 1   num_doc             698022 non-null  int64  
 2   nombre              698022 non-null  object 
 3   tipo_persona        698022 non-null  object 
 4   ingresos_mensuales  698022 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 32.0+ MB


In [143]:
transacciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360614 entries, 0 to 2360613
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   fecha_transaccion  object 
 1   cod_canal          float64
 2   tipo_doc           object 
 3   num_doc            int64  
 4   naturaleza         object 
 5   monto              float64
dtypes: float64(2), int64(1), object(3)
memory usage: 108.1+ MB


In [144]:
# Convertimos la columna de 'fecha_transaccion' de Object a datetime
transacciones['fecha_transaccion'] = pd.to_datetime(transacciones['fecha_transaccion'])

# Convertimos los columnas de datos correspondientes de Object a string
transacciones['tipo_doc'] = transacciones['tipo_doc'].astype("string")
transacciones['naturaleza'] = transacciones['naturaleza'].astype("string")
clientes['tipo_doc'] = clientes['tipo_doc'].astype("string")
clientes['nombre'] = clientes['nombre'].astype("string")
clientes['tipo_persona'] = clientes['tipo_persona'].astype("string")
canales['nombre'] = canales['nombre'].astype("string")
canales['tipo'] = canales['tipo'].astype("string")

# Convertimos la columna de double a int para eliminar los decimales
transacciones['cod_canal'] = transacciones['cod_canal'].astype(int)

# Convertimos las columnas que se pueden convertir numero a string
transacciones['num_doc'] = transacciones['num_doc'].astype("string")
transacciones['cod_canal'] = transacciones['cod_canal'].astype("string")
clientes['num_doc'] = clientes['num_doc'].astype("string")
canales['cod_jurisdiccion'] = canales['cod_jurisdiccion'].astype("string")
canales['codigo'] = canales['codigo'].astype("string")

# CARGA DE LOS DATOS

In [148]:
import pyodbc
from sqlalchemy import create_engine

server = 'LAPTOP-E360B96B\SQLEXPRESS'
database = 'Prueba_bancolombia'

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")

In [149]:
# Insertar datos en la tabla clientes
clientes.to_sql('clientes', engine, index=False, if_exists='replace')

# Insertar datos en la tabla canales
canales.to_sql('canales', engine, index=False, if_exists='replace')

# Insertar datos en la tabla transacciones
transacciones.to_sql('transacciones', engine, index=False, if_exists='replace')


327